In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [ ]:
from pyspark import SparkConf
configuration = SparkConf().setAppName('RomeoAndJulietCounter')\
                           .setMaster('local[*]')

In [ ]:
from pyspark import SparkContext
sc = SparkContext(conf=configuration)

In [ ]:
from textblob.utils import strip_punc
tokenized = sc.textFile('RomeoAndJuliet.txt')\
              .map(lambda line: strip_punc(line, all=True).lower())\
              .flatMap(lambda line: line.split())

In [ ]:
filtered = tokenized.filter(lambda word: word not in stop_words)

In [ ]:
from operator import add
word_counts = filtered.map(lambda word: (word, 1)).reduceByKey(add)

In [ ]:
filtered_counts = word_counts.filter(lambda item: item[1] >= 60)

In [ ]:
from operator import itemgetter
sorted_items = sorted(filtered_counts.collect(),
                      key=itemgetter(1), reverse=True)

In [ ]:
max_len = max([len(word) for word, count in sorted_items])
for word, count in sorted_items:
    print(f'{word:>{max_len}}: {count}')